In [135]:
import Fetcher
import transformers
from sentence_transformers import SentenceTransformer
from numpy import dot
from numpy.linalg import norm
import numpy as np
import random
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
import pandas as pd
from together import Together

def get_summaries(a):

    client = Together()
    sum_dict = {}

    for i in range(0,len(a)):
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.2-3B-Instruct-Turbo",
            messages=[{"role": "user", "content": a[i] + "Generate a bulleted list of main ideas. Do not add anything other than content"}],
        )
        topic = response.choices[0].message.content
        topic = topic.strip("Here's a bulleted list of the main ideas from the article: \n")

        response2 = client.chat.completions.create(
            model="meta-llama/Llama-3.2-3B-Instruct-Turbo",
            messages=[{"role": "user", "content": a[i] + "Generate a header based on the main ideas in 10 words or less."}],
        )
        header = response2.choices[0].message.content

        sum_dict[header] = topic
    return sum_dict


feed_url = 'https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml'
articles = Fetcher.scrape_news_from_feed(feed_url)

#stores a list of articles in a dictionary
article_dict = {}
for i in range(0, len(articles)):
    article_dict["article{}".format(i)] = articles[i]["content"]


#variables:

#arbirtrary threshold for similarity
threshold = 1.3
#list of articles to be clustered
list_of_articles = list(article_dict.values())

#sentence transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#list of vectors about sentence semantics for each sentence
embeddings = model.encode(list_of_articles) 

#links embeddings using ward method
Z = linkage(embeddings, method='ward')

# Create clusters by cutting the dendrogram at a certain threshold (t)
clusters = fcluster(Z, t=threshold, criterion='distance')

#creates a dictionary with numbered articles and their clustered
cluster_dict = {}
for i in range(0,len(clusters)):
    cluster_dict["article{}".format(i)] = clusters[i]

#initializes a dictionary w keys being each of the clusters 
grouped_dict = {}
for cluster_num in np.unique(list(cluster_dict.values())):
    grouped_dict[cluster_num] = []

#values of cluster_dict defined as all articles with same cluster in a list
for k,v in cluster_dict.items():
    if k not in grouped_dict:
        grouped_dict[v].append(k)


#removes all clusters with only one article
filtered_grouped_list = [] 
for k,v in grouped_dict.items():
    if len(v) > 1:
        filtered_grouped_list.append(v)

#creates list of articles where all articles of same topic are concatonated into one string
articles_to_summarize = []
for v in filtered_grouped_list:
    one_topic_list = []
    for art in v:
        one_topic_list.append((article_dict[art]))
    articles_to_summarize.append("\t".join(one_topic_list))

In [156]:
sum_dict = get_summaries(articles_to_summarize)

In [157]:
for header, topic in sum_dict.items():
    print(header)
    print(topic)
    print("\n")


Mayor Eric Adams Rejects 'Fascist' Label for Donald Trump
• Mayor Eric Adams of New York rejected the comparison of former President Donald J. Trump to Adolf Hitler, saying he should not be called a "fascist".
• The comments came as Trump was trying to make inroads with Black voters, especially Black men, in his campaign against Vice President Kamala Harris.
• Ms. Harris had previously agreed with Lt. Gen. John F. Kelly that Trump meets the definition of a fascist and praised Hitler.
• Mayor Adams was briefing reporters about security plans ahead of Trump's rally at Madison Square Garden.
• The mayor's record and the federal bribery and corruption charges he faces were also under scrutiny from Democrats running to unseat him.
• Attacks on Mayor Adams included criticism of his budget cuts, stalled bus improvements, and response to the city's affordability crisis.


"Trump's Dark Campaign Ends in Authoritarianism and Uncertainty"
• A campaign marked by Donald Trump's apocalyptic extremes